In [ ]:
import json
import os
import numpy as np
from pprint import pprint
from copy import deepcopy

import GraphPlotter
import Helper
from ReadParameterFile import get_parameter_dict
from NameMapper import map_name_from_file, map_name_from_json

In [ ]:
# Import the parameter file
params = get_parameter_dict('semiannually-params-json')
params

In [ ]:
"""
Prpares the input data to be plotted
"""
def prep_data(params, filename, language="en"):
    data = json.load(open(os.path.join(params["input_dir"], filename)))
    data = data["data"]
    
    # Preparation for party merging (child party to parent party)
    with open("../keywords/target/party.json") as fin:
        parties = json.load(fin)
        
    party_name_dict = map_name_from_json("../keywords/target/party.json", language=language)
    
    for k in data:
        raw_data = data[k]
    
        # Get length
        length = len(data[k][list(data[k].keys())[0]])

        # Cleanup data and produce overall values
        overall = np.zeros(length, dtype=np.int32)
        for r_k in raw_data:
            raw_data[r_k] = np.array(raw_data[r_k])
            overall += raw_data[r_k]
        data[k] = raw_data
        data[k]["overall"] = overall
        
        """
        for _k in data[k]:
            data[k][_k] = Helper.summarize_values_to_week(data[k][_k])
        """
    
    # Party merging
    for k in data:
        if k in party_name_dict and party_name_dict[k] in parties and parties[party_name_dict[k]]["parent_party"] is not None and party_name_dict[parties[party_name_dict[k]]["parent_party"]] != party_name_dict[k]:
            for r_k in data[k]:
                data[party_name_dict[parties[party_name_dict[k]]["parent_party"]]][r_k] += data[k][r_k]
    
    # Resize days to week
    for k in data:
        for _k in data[k]:
            data[k][_k] = Helper.summarize_values_to_week(data[k][_k])
        
    return data

In [ ]:
"""
Filter out the people/party that we need
"""
def filter_map(data, filter_list, map_file="name_mapping.csv", language="en"):
    # Get the map of leaders/parties
    #name_dict = map_name_from_file(map_file)
    name_dict = map_name_from_json("../keywords/target/leader.json", language=language)
    name_dict = map_name_from_json("../keywords/target/party.json", language=language, name_dict=name_dict)

    # Filter out people/party of interest
    ret_map = {}
    for name in filter_list:
        if name_dict[name] in data:
            ret_map[name_dict[name]] = data[name_dict[name]]
    
    return ret_map

In [ ]:
"""
Compilation of all parties that we want to plot
"""
def plot_helper(data, params, lang, name_map_file="name_mapping.csv"):
    date = params["start_year"] + "-" + params["start_month"] + " -- " + params["end_year"] + "-" + params["end_month"]
    
    short_lang = "en" if lang == "english" else "cn"
    
    cat_type = 'leader_different'
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "party_different"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Different coalition party "+ date, filtered_dict, "party_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party "+ date, filtered_dict, "party_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party "+ date, filtered_dict, "party_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_bn"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_same_bn"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "leader_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_malay"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_chinese"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "mca_v_dap"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "bn_v_ph_v_pas"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

In [ ]:
# Prepare the data
data = prep_data(params, "english_extracted.json")

# Plot the graph
plot_helper(data, params, "english")

In [ ]:
data = prep_data(params, "chinese_extracted.json")
plot_helper(data, params, "chinese")